# <center>Big Data &ndash; Exercise 1</center>
## <center>Fall 2023 &ndash; Week 1 &ndash; ETH Zurich</center>

### Aims
- **After this exercise:** 
    - Understand the SQL language and its common query patterns.
    - Understand the 'table' data shape, normalization, and when they can (and should) be used.
    - be able to query data in tables with the SQL language.
- **Later in the semester:** 
    - Relate these language features and query patterns relative to other data shapes, technologies, and the languages designed to query them.
    - Understand when tables are not the appropriate shape for your data and when you can (and should) throw normalization away!

### Prerequisites
In this exercise, you will brush-up the fundamental concepts of relational databases and SQL. If you haven't taken an introductory databases course (or want to refresh your knowledge) we recommend you to read the following:

Garcia-Molina, Ullman, Widom: Database Systems: The Complete Book. Pearson, 2. Edition, 2008. (Chapters 1, 2, 3, and 6) [Available in the ETH Library] [[Online]](https://ebookcentral.proquest.com/lib/ethz/detail.action?pq-origsite=primo&docID=5832965) [[Selected solutions]](http://infolab.stanford.edu/~ullman/dscbsols/sols.html).

Or have a look at the recordings from Information Systems for Engineers - ETH Zurich, available on [[YouTube]](https://www.youtube.com/c/GhislainFournysLectures).

### Database Set-up
We will be once again working in the ExamMagicBox (you can find it in the following [[link]](http://please.add.link.here) if you have not downloaded it yet): please drag this Notebook in the folder. Just like last week, activate the docker container for the exercise sheet with `docker compose up`; please wait for the message `PostgreSQL init process complete; ready for start up` in the docker logs before proceeding! Alternatively you can start the Docker with `docker compose up -d` and wait for the command to execute: please note that you are creating the containers in the background this way. You can then type `docker compose down` when you are done.

As before, we set up our connection to the database and enable use of `%sql` and `%%sql`.

In [1]:
server='db'
user='postgres'
password='example'
database='postgres'
connection_string=f'postgresql://{user}:{password}@{server}:5432/{database}'

In [2]:
%reload_ext sql
%sql $connection_string

In [3]:
%%sql
SELECT version();

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


version
"PostgreSQL 16.0 (Debian 16.0-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit"


### Origin of the data
You can find more information on the dataset in the following links
- [Discogs](https://www.discogs.com/)
- [Discogs XML data dumps](http://data.discogs.com/)

If you do not want to use Docker or it does not work you can download the dataset from this [link](https://cloud.inf.ethz.ch/s/DtjCHTLRHT39BRN/download/discogs.dump.xz), see `postgres-init.sh` to see how to import it)

## Exercise 1: Explore the dataset
We want to first understand the dataset a bit better. You will find some queries below to help you explore the schema.

### List tables
The following query retrieves a list of tables in the database from a system table describing the current database.

In [4]:
%%sql 
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';

 * postgresql://postgres:***@db:5432/postgres
22 rows affected.


table_name
companies
jobs
employees02
badges
comments
inventory
posthistory
postlinks
posts
tags


### List attributes/columns
The following query retrieves a list of columns from the tables in the database we will be using.

In [5]:
%%sql 
SELECT table_name, column_name, data_type, is_nullable, ordinal_position
FROM information_schema.columns
WHERE table_schema = 'public' AND table_name IN ('artists', 'released_by', 'releases', 'tracks')
AND table_name NOT LIKE 'pg_%'
ORDER BY table_name, ordinal_position;

 * postgresql://postgres:***@db:5432/postgres
17 rows affected.


table_name,column_name,data_type,is_nullable,ordinal_position
artists,artist_id,integer,NO,1
artists,name,character varying,YES,2
artists,realname,text,YES,3
artists,profile,text,YES,4
artists,url,text,YES,5
released_by,release_id,integer,NO,1
released_by,artist_id,integer,NO,2
releases,release_id,integer,NO,1
releases,released,date,NO,2
releases,title,text,NO,3


###  List the titles of all releases by Coldplay that contain less than 5 tracks.

In [16]:
%%sql
SELECT DISTINCT releases.title 
FROM artists JOIN released_by USING(artist_id)
JOIN releases USING(release_id) 
JOIN tracks USING(release_id)
WHERE artists.name = 'Coldplay'
GROUP BY release_id, releases.title
HAVING COUNT(*) < 5;

 * postgresql://postgres:***@db:5432/postgres
34 rows affected.


title
Acoustic
Boot Of Sound
Brothers & Sisters
Clocks
Clocks...
Clocks / Chime Trance Remixes
Clocks (Cosmos Rmx)
Clocks (Dean Coleman Remix)
Clocks (Planet Rockers Remixes)
Clocks (Remix)


### What are the top 10 artists with the most releases?

In [17]:
%%sql
select artists.name
FROM artists JOIN released_by USING(artist_id)
GROUP BY released_by.artist_id, artists.name
ORDER BY COUNT(*) DESC
LIMIT 10;

 * postgresql://postgres:***@db:5432/postgres
10 rows affected.


name
Various Artists
Depeche Mode
Madonna
Pet Shop Boys
Moby
New Order
Erasure
The Cure
U2
The Prodigy


### How many artists have more releases than the average number of releases per artist? Please ignore artists with 0 releases.

In [21]:
%%sql
WITH rpa AS (SELECT artist_id, COUNT(*) AS releases
            FROM released_by
            GROUP BY artist_id)
SELECT COUNT(*)
FROM rpa
WHERE releases > (SELECT AVG(releases)
                 FROM rpa);


 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


count
23632


### Have a look at the datasets
The following simple query gives the first 5 rows of the `artists` dataset

In [ ]:
%%sql
SELECT * FROM artists LIMIT 5;

Naturally we could write similar queries to better understand each of the other tables.

### With what you now know about the datasets, try to answer the following questions

1. Which concepts are modelled in the dataset and how do they relate to each other? <b>Hint</b>: how do the tables connect logically?
2. Why do you think this shape (table) was chosen for the data and why not the other shapes?
3. In which normal forms are the corresponding relations?
4. How can we denormalise the data to make some queries more efficient? <b>Hint</b>: have a look at the queries in the next session of the exercises to see if adding some columns to some tables could reduce the need to `JOIN`.
5. What potential problems could result from adding redundancy?

## Exercise 2: SQL warm-up
Let us begin with several SQL queries to ease us back into the language.

#### 1. Retrieve the artists with the name of 'Coldplay'.

#### 2. List the titles of all releases by that artist in alphabetical order.
<b>Hint</b>: Ignore the fact that different relases can have the same title.

#### 3. How many tracks from 'Coldplay' have position '1'?

#### 4. What is the average track duration?

#### 5. How many artists have released tracks longer than twice the average?

## Exercise 3: more SQL
We will now see more complex SQL queries.

#### 1. What is the title of the album from 'Coldplay' with the most amount of tracks?

#### 2. How many artists have released an album with total track duration above twice the average total track duration?

<b>Hint</b>: this is not the same as exercise 2.5 since we are lookong at the <b>total</b> track duration of the album.

#### 3. How many artists have both a release with a track longer than twice the average and one with total duration longer than twice the average?

<b>Hint</b>: you can use `INTERSECT` or `EXISTS` to write your query.

#### 4. Show the artists have more than 200 releases in total but have no releases with the genre 'Pop' in reversed alphabetical order.

## Exercise 4: Discuss query patterns and language features of SQL
1. What patterns did you use in many of the queries above? 

2. What is the usual pattern of an SQL query? Which operations happen pre-grouping and which ones post-grouping?

3. What makes SQL a declarative language and what advantages does that have?

4. What makes SQL a functional language and what advantages does that have?

5. How would the denormalization we talked about previously simplify the queries?

## Exercise 5: Limits of SQL (optional)
Explain what the following query does.
<b>Hints</b>: The query treats the data as if it was in graph shape.

In [ ]:
%%sql
WITH RECURSIVE
    X AS (SELECT 3 AS Value),
    artist_releases AS (
        SELECT artists.artist_id, artists.name, releases.release_id, releases.title
        FROM artists, released_by, releases
        WHERE artists.artist_id = released_by.artist_id
        AND released_by.release_id = releases.release_id
    ),
    collaborations AS (
        SELECT DISTINCT ar1.artist_id AS left_id, ar1.name AS left_name, 
                ar2.artist_id AS right_id, ar2.name AS right_name, 1 AS distance
        FROM artist_releases AS ar1, artist_releases AS ar2
        WHERE ar1.release_id = ar2.release_id
        AND ar1.artist_id != ar2.artist_id
    ),
    X_hop_collaborations AS (
        SELECT * FROM collaborations  -- base case
        UNION
        SELECT c1.left_id, c1.left_name, c2.right_id, c2.right_name, c1.distance + 1 AS distance
        FROM X_hop_collaborations AS c1
        JOIN collaborations c2 ON c1.right_id = c2.left_id
        WHERE c1.distance < (SELECT * FROM X) AND c1.left_id != c2.right_id
    )
SELECT * 
FROM X_hop_collaborations
WHERE left_name = 'Coldplay'
ORDER BY distance, right_name;